<a href="https://colab.research.google.com/github/marcojrr/Projetos/blob/main/analise_lol_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("Introdução à análise de dados com PySpark utilizando os dados dos campeões do LOL")
        .getOrCreate()
        )

In [5]:
import requests

response=requests.get(
"https://ddragon.leagueoflegends.com/cdn/12.17.1/data/pt_BR/champion.json")

champions=response.json().get("data")
champions.keys()

dict_keys(['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Belveth', 'Blitzcrank', 'Brand', 'Braum', 'Caitlyn', 'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy', 'Pyke', 'Qiyana', 'Quinn', 'Rakan', 'Ramm

In [6]:
champions.get('Akali')

{'version': '12.17.1',
 'id': 'Akali',
 'key': '84',
 'name': 'Akali',
 'title': 'a Assassina Renegada',
 'blurb': 'Abandonando a Ordem Kinkou e seu título de Punho das Sombras, Akali agora ataca sozinha, pronta para ser a arma mortal que seu povo precisa. Embora ela mantenha tudo o que aprendeu com seu mestre Shen, ela se comprometeu a defender Ionia de seus...',
 'info': {'attack': 5, 'defense': 3, 'magic': 8, 'difficulty': 7},
 'image': {'full': 'Akali.png',
  'sprite': 'champion0.png',
  'group': 'champion',
  'x': 96,
  'y': 0,
  'w': 48,
  'h': 48},
 'tags': ['Assassin'],
 'partype': 'Energia',
 'stats': {'hp': 570,
  'hpperlevel': 119,
  'mp': 200,
  'mpperlevel': 0,
  'movespeed': 345,
  'armor': 23,
  'armorperlevel': 4.7,
  'spellblock': 37,
  'spellblockperlevel': 2.05,
  'attackrange': 125,
  'hpregen': 9,
  'hpregenperlevel': 0.9,
  'mpregen': 50,
  'mpregenperlevel': 0,
  'crit': 0,
  'critperlevel': 0,
  'attackdamage': 62,
  'attackdamageperlevel': 3.3,
  'attackspeedpe

In [7]:
champions = [{'name': value['name'], 'title': value['title'], **value['info'], **value['stats']} for key, value in champions.items()]
champions[2]

{'name': 'Akali',
 'title': 'a Assassina Renegada',
 'attack': 5,
 'defense': 3,
 'magic': 8,
 'difficulty': 7,
 'hp': 570,
 'hpperlevel': 119,
 'mp': 200,
 'mpperlevel': 0,
 'movespeed': 345,
 'armor': 23,
 'armorperlevel': 4.7,
 'spellblock': 37,
 'spellblockperlevel': 2.05,
 'attackrange': 125,
 'hpregen': 9,
 'hpregenperlevel': 0.9,
 'mpregen': 50,
 'mpregenperlevel': 0,
 'crit': 0,
 'critperlevel': 0,
 'attackdamage': 62,
 'attackdamageperlevel': 3.3,
 'attackspeedperlevel': 3.2,
 'attackspeed': 0.625}

In [17]:
import pandas as pd


df = spark.createDataFrame(pd.DataFrame(champions))

df.select('name', 'title').show(10, False)

+--------+-----------------------+
|name    |title                  |
+--------+-----------------------+
|Aatrox  |a Espada Darkin        |
|Ahri    |a Raposa de Nove Caudas|
|Akali   |a Assassina Renegada   |
|Akshan  |o Sentinela Rebelde    |
|Alistar |o Minotauro            |
|Amumu   |a Múmia Triste         |
|Anivia  |a Criofênix            |
|Annie   |a Criança Sombria      |
|Aphelios|a Arma dos Devotos     |
|Ashe    |a Arqueira do Gelo     |
+--------+-----------------------+
only showing top 10 rows



In [19]:
from pyspark.sql import functions as F

df = df.withColumn('full_name', F.concat(df.name, F.lit(', '), df.title))

df.select('full_name').show(10, False)

+-----------------------------+
|full_name                    |
+-----------------------------+
|Aatrox, a Espada Darkin      |
|Ahri, a Raposa de Nove Caudas|
|Akali, a Assassina Renegada  |
|Akshan, o Sentinela Rebelde  |
|Alistar, o Minotauro         |
|Amumu, a Múmia Triste        |
|Anivia, a Criofênix          |
|Annie, a Criança Sombria     |
|Aphelios, a Arma dos Devotos |
|Ashe, a Arqueira do Gelo     |
+-----------------------------+
only showing top 10 rows



In [20]:
base_columns = ['attackdamage', 'armor', 'hp', 'mp']

(df.orderBy(*base_columns, ascending=False)
  .select('full_name', *base_columns)
  .show(10, False)
  )

+---------------------------------+------------+-----+-----+-----+
|full_name                        |attackdamage|armor|hp   |mp   |
+---------------------------------+------------+-----+-----+-----+
|Tryndamere, o Rei Bárbaro        |72.0        |33   |696.0|100.0|
|Cho'Gath, o Terror do Vazio      |69.0        |38   |644.0|270.0|
|Renekton, o Carniceiro das Areias|69.0        |35   |660.0|100.0|
|Ornn, O Fogo sob a Montanha      |69.0        |33   |660.0|340.6|
|Kayn, o Ceifador das Sombras     |68.0        |38   |655.0|410.0|
|Trundle, o Rei dos Trolls        |68.0        |37   |686.0|281.0|
|Jax, o Grão-Mestre das Armas     |68.0        |36   |685.0|339.0|
|Lee Sin, o Monge Cego            |68.0        |36   |645.0|200.0|
|Illaoi, a Sacerdotisa Cráquem    |68.0        |35   |656.0|300.0|
|Camille, a Sombra de Aço         |68.0        |35   |646.0|339.0|
+---------------------------------+------------+-----+-----+-----+
only showing top 10 rows



In [21]:
level = 10

df2 = df.withColumns({
    'attackdamage': df.attackdamage + df.attackdamageperlevel * level,
    'armor': df.armor + df.armorperlevel * level,
    'hp': df.hp + df.hpperlevel * level,
    'mp': df.mp + df.mpperlevel * level
})

In [23]:
(df2.orderBy(*base_columns, ascending=False)
 .select('full_name', *base_columns)
 .show(10, False)
 )

+---------------------------------+------------+-----+------+-----+
|full_name                        |attackdamage|armor|hp    |mp   |
+---------------------------------+------------+-----+------+-----+
|Illaoi, a Sacerdotisa Cráquem    |118.0       |85.0 |1746.0|800.0|
|Olaf, o Berserker                |115.0       |77.0 |1835.0|816.0|
|Darius, a Mão de Noxus           |114.0       |91.0 |1792.0|838.0|
|Yorick, o Pastor de Almas        |112.0       |91.0 |1790.0|900.0|
|Cho'Gath, o Terror do Vazio      |111.0       |85.0 |1584.0|870.0|
|Garen, o Poder de Demacia        |111.0       |78.0 |1670.0|0.0  |
|Renekton, o Carniceiro das Areias|110.5       |87.0 |1770.0|100.0|
|Jax, o Grão-Mestre das Armas     |110.5       |78.0 |1675.0|859.0|
|Skarner, a Vanguarda de Cristal  |110.0       |88.0 |1640.0|920.0|
|Aatrox, a Espada Darkin          |110.0       |82.5 |1790.0|0.0  |
+---------------------------------+------------+-----+------+-----+
only showing top 10 rows



In [24]:
level = 18

df3 = df.withColumns({
    'attackdamage': df.attackdamage + df.attackdamageperlevel * level,
    'armor': df.armor + df.armorperlevel * level,
    'hp': df.hp + df.hpperlevel * level,
    'mp': df.mp + df.mpperlevel * level
})

In [26]:
(df3.orderBy(*base_columns, ascending=False)
 .select('full_name', *base_columns)
 .show(10, False)
 )

+---------------------------------+------------------+------------------+------+------+
|full_name                        |attackdamage      |armor             |hp    |mp    |
+---------------------------------+------------------+------------------+------+------+
|Illaoi, a Sacerdotisa Cráquem    |158.0             |125.0             |2618.0|1200.0|
|Darius, a Mão de Noxus           |154.0             |132.60000000000002|2704.0|1298.0|
|Olaf, o Berserker                |152.60000000000002|110.60000000000001|2787.0|1216.0|
|Yorick, o Pastor de Almas        |152.0             |132.60000000000002|2702.0|1380.0|
|Aatrox, a Espada Darkin          |150.0             |118.10000000000001|2702.0|0.0   |
|Garen, o Poder de Demacia        |147.0             |111.60000000000001|2454.0|0.0   |
|Skarner, a Vanguarda de Cristal  |146.0             |128.0             |2432.0|1400.0|
|Cho'Gath, o Terror do Vazio      |144.60000000000002|122.60000000000001|2336.0|1350.0|
|Jax, o Grão-Mestre das Armas   

In [27]:
(df2.agg({
    'attackdamage': 'mean',
    'hp': 'max',
    'mp': 'max',
    'armor': 'min'
})
    .show()
 )

+-------+----------+-----------------+-------+
|max(mp)|min(armor)|avg(attackdamage)|max(hp)|
+-------+----------+-----------------+-------+
|10000.0|      28.0|91.40481987577641| 1892.0|
+-------+----------+-----------------+-------+



In [31]:
df2.filter(df2.mp == 10000).show()

+-----+---------------+------+-------+-----+----------+------+----------+-------+----------+---------+-----+-------------+----------+------------------+-----------+-------+---------------+-------+---------------+----+------------+------------+--------------------+-------------------+-----------+--------------------+
| name|          title|attack|defense|magic|difficulty|    hp|hpperlevel|     mp|mpperlevel|movespeed|armor|armorperlevel|spellblock|spellblockperlevel|attackrange|hpregen|hpregenperlevel|mpregen|mpregenperlevel|crit|critperlevel|attackdamage|attackdamageperlevel|attackspeedperlevel|attackspeed|           full_name|
+-----+---------------+------+-------+-----+----------+------+----------+-------+----------+---------+-----+-------------+----------+------------------+-----------+-------+---------------+-------+---------------+----+------------+------------+--------------------+-------------------+-----------+--------------------+
|Viego|O Rei Destruído|     6|      4|    2|  